## Here, I attempt to make an `openmm.CustomIntegrator` whilst respecting much of the `openmmtools.integrators` codebase.

In [1]:
from openmmtools import testsystems
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion

from coddiwomple.openmm.propagators import *
from coddiwomple.openmm.reporters import *
from coddiwomple.openmm.utils import *
from coddiwomple.openmm.states import *

from simtk import unit
import copy
import numpy as np
# from openmmtools.states import *
# from openmmtools.alchemy import AlchemicalState
# from openmmtools.integrators import *


RDKit WARNING: [00:08:16] Enabling RDKit 2019.09.3 jupyter extensions


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [2]:
custom_functions = {'lambda_sterics': f"fractional_iteration",
                    'lambda_electrostatics': f"step(fractional_iteration - 0.5) * (fractional_iteration - 0.5) * 2.0"}
new_integrator = OpenMMIAAISLangevinIntegrator(custom_functions, 100)


DEBUG:openmm_propagators:initializing OpenMMIAAISLangevinIntegrator...
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: metropolization is False
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: measure_proposal_work is False
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: measure_shadow_work is False
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: successfully parsed splitting string
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: adding global variables...
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: adding integrator steps...
DEBUG:openmm_propagators:OpenMMIAAISLangevinIntegrator: adding substep functions...


In [3]:
ala_vac_sys = testsystems.AlanineDipeptideVacuum().system
factory = AbsoluteAlchemicalFactory(consistent_exceptions = False)
alchemical_region = AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=ala_vac_sys,
                                                            alchemical_regions = alchemical_region)

DEBUG:openmmtools.alchemy:Dictionary of interacting alchemical regions: frozenset()
DEBUG:openmmtools.alchemy:Using 1 alchemical regions
DEBUG:openmmtools.alchemy:Adding steric interaction groups between  and the environment.
DEBUG:openmmtools.alchemy:Adding a steric interaction group between group  and .
DEBUG:openmmtools.alchemy:Adding electrostatic interaction groups between  and the environment.
DEBUG:openmmtools.alchemy:Adding a electrostatic interaction group between group  and .
DEBUG:openmmtools.utils:Create alchemically modified system took    0.043s


In [4]:
pdf_state = OpenMMPDFState(system = alanine_alchemical_system, pressure = None)
propagator = OpenMMBaseIntegrationPropagator(openmm_pdf_state = pdf_state, integrator = new_integrator)


DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_torsions': None, 'lambda_electrostatics': 1.0, 'lambda_angles': None, 'lambda_sterics': 1.0, 'lambda_bonds': None}
DEBUG:openmm_propagators:successfully executed ABCMeta init.
DEBUG:openmm_propagators:successfully equipped integrator: OpenMMIAAISLangevinIntegrator
DEBUG:openmm_propagators:integrator printable: None


step      0 : allow forces to update the context state
step      1 : if(has_kT_changed = 1):
step      2 :    sigma <- sqrt(kT/m)
step      3 :    has_kT_changed <- 0
step      4 : end
step      5 : Uold <- energy
step      6 : iteration <- iteration + 1.0
step      7 : fractional_iteration <- iteration / niterations
step      8 : lambda_sterics <- fractional_iteration
step      9 : lambda_electrostatics <- step(fractional_iteration - 0.5) * (fractional_iteration - 0.5) * 2.0
step     10 : Unew <- energy
step     11 : state_work <- state_work + (Unew - Uold)
step     12 : v <- v + (dt / 2) * f / m
step     13 : constrain velocities
step     14 : x <- x + ((dt / 2) * v)
step     15 : x1 <- x
step     16 : constrain positions
step     17 : v <- v + ((x - x1) / (dt / 2))
step     18 : constrain velocities
step     19 : v <- (a * v) + (b * sigma * gaussian)
step     20 : constrain velocities
step     21 : x <- x + ((dt / 2) * v)
step     22 : x1 <- x
step     23 : constrain positions
step 

In [5]:
sampler_state = OpenMMParticleState(testsystems.AlanineDipeptideVacuum().positions)


In [6]:
params = propagator.context.getParameters()
for q in params:
    print(q, params[q])
    
global_vars = propagator.integrator.getNumGlobalVariables()
for i in range(global_vars):
    print(propagator.integrator.getGlobalVariableName(i), propagator.integrator.getGlobalVariable(i))

lambda_electrostatics 1.0
lambda_sterics 1.0
softcore_a 1.0
softcore_alpha 0.5
softcore_b 1.0
softcore_beta 0.0
softcore_c 6.0
softcore_d 1.0
softcore_e 1.0
softcore_f 2.0
_restorable__class_hash 2600602343.0
kT 2.4943417413660645
a 0.999000499833375
b 0.044699008184376096
proposal_work 0.0
old_ke 0.0
new_ke 0.0
state_work 0.0
Uold 0.0
Unew 0.0
niterations 100.0
iteration 0.0
fractional_iteration 0.0
has_kT_changed 1.0


In [7]:
propagator.apply(sampler_state, n_steps = 100)

(<coddiwomple.openmm.states.OpenMMParticleState at 0x7fb624f3f240>,
 (0.0, -0.635376073054278))

In [8]:
params = propagator.context.getParameters()
for q in params:
    print(q, params[q])
    
global_vars = propagator.integrator.getNumGlobalVariables()
for i in range(global_vars):
    print(propagator.integrator.getGlobalVariableName(i), propagator.integrator.getGlobalVariable(i))

lambda_electrostatics 1.0
lambda_sterics 1.0
softcore_a 1.0
softcore_alpha 0.5
softcore_b 1.0
softcore_beta 0.0
softcore_c 6.0
softcore_d 1.0
softcore_e 1.0
softcore_f 2.0
_restorable__class_hash 2600602343.0
kT 2.4943417413660645
a 0.999000499833375
b 0.044699008184376096
proposal_work 0.0
old_ke 0.0
new_ke 0.0
state_work -1.5848450604845397
Uold -107.047257063261
Unew -109.72281025561891
niterations 100.0
iteration 100.0
fractional_iteration 1.0
has_kT_changed 0.0


In [9]:
propagator.apply(sampler_state, n_steps = 99)

(<coddiwomple.openmm.states.OpenMMParticleState at 0x7fb624f3f240>,
 (0.0, -111.89431140650005))

In [10]:
params = propagator.context.getParameters()
for q in params:
    print(q, params[q])
    
global_vars = propagator.integrator.getNumGlobalVariables()
for i in range(global_vars):
    print(propagator.integrator.getGlobalVariableName(i), propagator.integrator.getGlobalVariable(i))

lambda_electrostatics 2.98
lambda_sterics 1.99
softcore_a 1.0
softcore_alpha 0.5
softcore_b 1.0
softcore_beta 0.0
softcore_c 6.0
softcore_d 1.0
softcore_e 1.0
softcore_f 2.0
_restorable__class_hash 2600602343.0
kT 2.4943417413660645
a 0.999000499833375
b 0.044699008184376096
proposal_work 0.0
old_ke 0.0
new_ke 0.0
state_work -279.10265156264603
Uold -372.00378751352633
Unew -374.87358317936906
niterations 100.0
iteration 199.0
fractional_iteration 1.99
has_kT_changed 0.0


In [ ]:
print(integrator.global_variable_names)
print(integrator.getTemperature())
print(integrator.kT)
q = context.getParameters()
for x in q:
    print(x, q[x])



In [ ]:
#build a target/proposal factory
propagator = OpenMMLangevinDynamicsSplittingMove(n_steps = 1)
elecs = np.append(np.linspace(1.0, 0.0, 50), np.zeros(50))
sterics = np.linspace(1.0, 0.0, 100)
parameter_sequence = [{'lambda_electrostatics': i, 'lambda_sterics': j} for i,j in zip(elecs, sterics)]
proposal_factory = OpenMMProposalFactory(proposal_pdf_state, parameter_sequence, propagator)

In [ ]:
propagator.pretty_print()